# Preprocessing and Training Data Development - Football Transfer Evaluation

We will continue the work in the previous notebook and look to produce the features that we will use in creating our model. In order to get all our features set, we need to identify our categorical data vs. numerical data and create a clean, useable data set for our model. We start by having a look again at our data, found in the 